In [1]:
import pandas as pd
df = pd.read_parquet('./data/catB_train.parquet')

In [2]:
df = df[df['cltdob_fix']!='None']
df['cltdob_fix'] = pd.to_datetime(df.iloc[:, 6], format ='mixed')
df['age'] = 2024-df['cltdob_fix'].dt.year

df["f_purchase_lh"] = df["f_purchase_lh"].fillna(0)
y = df["f_purchase_lh"]
X = df.drop(columns=['f_purchase_lh'])

numeric_cols = X.select_dtypes(include=["int32", "int64", "float64"]).columns
X_numeric = X[numeric_cols]
X_numeric = X_numeric.apply(lambda x: x.fillna(-1))

from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=(0.05))
sel.fit(X_numeric)
X_numeric = X_numeric[numeric_cols[sel.get_support()]]

non_numeric_cols = X.select_dtypes(include=["string", "object"]).columns
non_numeric_keep = []
for col in non_numeric_cols:
    if len(X[col].unique()) <= 5:
        if len(X[col].unique()) > 1:
            non_numeric_keep.append(col)
X_non_numeric = pd.get_dummies(X[non_numeric_keep], dtype=float)

X = pd.concat([X_numeric, X_non_numeric], axis=1)

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

In [4]:
from imblearn.over_sampling import RandomOverSampler, SMOTE

ros = RandomOverSampler(random_state=0)
X_ros, y_ros = ros.fit_resample(X_train, y_train)
smote = SMOTE(random_state=0)
X_smote, y_smote = smote.fit_resample(X_train, y_train)

In [5]:
from sklearn.ensemble import HistGradientBoostingClassifier
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

bbc_ros = BalancedBaggingClassifier(estimator=HistGradientBoostingClassifier(random_state=0), n_estimators=15, random_state=0)
bbc_smote = BalancedBaggingClassifier(estimator=HistGradientBoostingClassifier(random_state=0), n_estimators=15, random_state=0)

In [6]:
bbc_smote.fit(X_smote, y_smote)
y_val_pred_bbc = bbc_smote.predict(X_val)
conf_matrix_bbc = confusion_matrix(y_val, y_val_pred_bbc)
print("\nBalanced Bagging Classifier with SMOTE Results:")
print(conf_matrix_bbc)
print(f"Accuracy: {accuracy_score(y_val, y_val_pred_bbc)}")
print(f"Precision: {precision_score(y_val, y_val_pred_bbc)}")
print(f"Recall: {recall_score(y_val, y_val_pred_bbc)}")
print(f"F1 Score: {f1_score(y_val, y_val_pred_bbc)}")


Balanced Bagging Classifier with SMOTE Results:
[[3415   33]
 [ 117   29]]
Accuracy: 0.9582637729549248
Precision: 0.46774193548387094
Recall: 0.19863013698630136
F1 Score: 0.27884615384615385


In [7]:
bbc_ros.fit(X_ros, y_ros)
y_val_pred_ros = bbc_ros.predict(X_val)
conf_matrix_ros = confusion_matrix(y_val, y_val_pred_ros)
print("\nBalanced Bagging Classifier with ROS Results:")
print(conf_matrix_ros)
print(f"Accuracy: {accuracy_score(y_val, y_val_pred_ros)}")
print(f"Precision: {precision_score(y_val, y_val_pred_ros)}")
print(f"Recall: {recall_score(y_val, y_val_pred_ros)}")
print(f"F1 Score: {f1_score(y_val, y_val_pred_ros)}")


Balanced Bagging Classifier with ROS Results:
[[3160  288]
 [  71   75]]
Accuracy: 0.9001112966054535
Precision: 0.2066115702479339
Recall: 0.5136986301369864
F1 Score: 0.29469548133595286
